In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.datasets import fetch_california_housing

In [ ]:
# 필요한 패키지들과 데이터셋을 불러오고, 판다스에서 읽어들여 Target이라는 이름을 갖는 열에 출력 데이터를 넣어준다.
california = fetch_california_housing()

df = pd.DataFrame(california.data, columns=california.feature_names)
df["Target"] = california.target
df.tail()

In [ ]:
# 표준 스케일링을 적용하여 입력 데이터를 정규화한다.
scaler = StandardScaler()
scaler.fit(df.values[:, :-1])
df.values[:, :-1] = scaler.transform(df.values[:, :-1]) # scaler.transform은 scaler.fit으로 학습된 변환을 데이터에 적용하는 부분이다.

df.tail()

In [ ]:
# 학습 코드 구현

In [ ]:
# 학습에 필요한 파이토치 관련 패키지 불러오기
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [ ]:
# 넘파이로 되어 있는 데이터셋을 파이토치 텐서로 변환한다.
data = torch.from_numpy(df.values).float()

x = data[:, :-1]
y = data[:, -1:] # 전체 행을 가져오고, 마지막 열만 가져오는 코드이다. -1은 마지막 열부터 끝까지 선택한다.

print(x.shape, y.shape)

In [ ]:
# 학습에 필요한 세팅 값을 선언한다.
# 학습률은 이제 필요없으니 주석처리한다.
n_epochs = 4000
batch_size = 256
print_interval = 200
#learning_rate = 1e-2

In [ ]:
# nn.Sequential을 활용하여 똑같이 모델을 만든다.
model = nn.Sequential( # 순차적으로 레이어를 추가하는 방법이다.
    nn.Linear(x.size(-1), 6),
    nn.LeakyReLU(),
    nn.Linear(6, 5),
    nn.LeakyReLU(),
    nn.Linear(5, 4),
    nn.LeakyReLU(),
    nn.Linear(4, 3),
    nn.LeakyReLU(),
    nn.Linear(3, y.size(-1)),
)

In [ ]:
# 만든 모델을 옵티마이저에 등록한다.
# 앞에서는 optim.SGD 클래스를 통해, 모델을 학습하기 위한 옵티마이저 객체를 생성했다.
# 이번에는 optim.Adam 클래스를 통해, 아담 옵티마이저를 선언한다.
# 이때, 모델 파라미터만 등록하되, 학습률은 인자로 넣어주지 않는 모습을 볼 수 있다.
optimizer = optim.Adam(model.parameters())

In [ ]:
# 이중 for문을 사용하여, 학습을 진행하는 코드이다.

# |x| = (total_size, input_dim)
# |y| = (total_size, output_dim)

for i in range(n_epochs):
    # Shuffle the index to feed-forward.
    indices = torch.randperm(x.size(0))
    x_ = torch.index_select(x, dim=0, index=indices)
    y_ = torch.index_select(y, dim=0, index=indices)

    x_ = x_.split(batch_size, dim=0)
    y_ = y_.split(batch_size, dim=0)
    # |x_[i]| = (batch_size, input_dim)
    # |y_[i]| = (batch_size, output_dim)

    y_hat = []
    total_loss = 0

    for x_i, y_i in zip(x_, y_):
        # |x_i| = |x_[i]|
        # |y_i| = |y_[i]|
        y_hat_i = model(x_i)
        loss = F.mse_loss(y_hat_i, y_i)

        optimizer.zero_grad()
        loss.backward()

        optimizer.step()

        total_loss += float(loss)
        y_hat += [y_hat_i]

    total_loss = total_loss / len(x_)
    if (i + 1) % print_interval == 0:
        print("Epoch %d: loss=%.4e" % (i + 1, total_loss))

y_hat = torch.cat(y_hat, dim=0)
y = torch.cat(y_, dim=0)
# |y_hat| = (total_size, output_dim)
# |y| = (total_size, output_dim)

In [ ]:
# 결과 확인
df = pd.DataFrame(torch.cat([y, y_hat], dim=1).detach().numpy(), columns=["y", "y_hat"])
sns.pairplot(df, height=5)
plt.show()